In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from pathlib import Path

import os
import sys

import numpy as np
import torch

sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'models'))

import ai8x
from datasets import nilm

In [2]:
data_path = Path('../data/NILM')
args = None
# data_path.exists()

train_set, test_set, val_set = nilm.ukdale_get_datasets((data_path, args), load_train=True, load_test=True, load_val=True)

In [3]:
# batch_size = 32
# num_workers = 5

# dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)

# batch_select = None
# for batch_idx, batch in enumerate(dataloader):
#     if batch_idx == 0:
#         batch_select = batch
#         break

In [6]:
batch_size = 128
num_workers = 5

train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)

max_in_batch = 0
min_in_batch = 100

train_batch_with_max = 0
test_batch_with_max = 0
val_batch_with_max = 0

batch_select = None

for batch_idx, batch in enumerate(train_dataloader):
        if batch[0].max() > max_in_batch:
                max_in_batch = batch[0].max()
                train_batch_with_max = batch_idx
                batch_select = batch

        if batch[0].min() < min_in_batch:
                min_in_batch = batch[0].min()

        if batch[1].max() > max_in_batch:
                max_in_batch = batch[1].max()

        if batch[1].min() < min_in_batch:
                min_in_batch = batch[1].min()

for batch_idx, batch in enumerate(test_dataloader):
        if batch[0].max() > max_in_batch:
                max_in_batch = batch[0].max()

        if batch[0].min() < min_in_batch:
                min_in_batch = batch[0].min()

        if batch[1].max() > max_in_batch:
                max_in_batch = batch[1].max()

        if batch[1].min() < min_in_batch:
                min_in_batch = batch[1].min()

for batch_idx, batch in enumerate(val_dataloader):
        if batch[0].max() > max_in_batch:
                max_in_batch = batch[0].max()
        
        if batch[0].min() < min_in_batch:
                min_in_batch = batch[0].min()

        if batch[1].max() > max_in_batch:
                max_in_batch = batch[1].max()

        if batch[1].min() < min_in_batch:
                min_in_batch = batch[1].min()

print(max_in_batch)
print(min_in_batch)
print(train_batch_with_max)


tensor(12.9169)
tensor(-1.)
2989


In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [8]:
ai8x.set_device(device=85, simulate=False, round_avg=False)

Configuring device: MAX78000, simulate=False.


In [9]:
quantiles = [0.0025,0.1, 0.5, 0.9, 0.975]

In [13]:
class Conv1D(nn.Module):
    
    def __init__(self,
                 num_channels,
                 num_kernels,
                 kernel_size=3,
                 stride=1,
                 padding=1,
                 last=False,
                 activation="ReLU",
                 batchnorm="NoAffine",
                 device="cuda:0",
                 **kwargs):
        super(Conv1D, self).__init__()
        
        if not last:
            if activation == "ReLU":
                self.net = ai8x.FusedConv1dBNReLU(in_channels=num_channels,
                                                  out_channels=num_kernels,
                                                  kernel_size=kernel_size,
                                                  stride=stride,
                                                  padding=padding,
                                                  bias=True,
                                                  batchnorm="NoAffine",
                                                  **kwargs)
            elif activation == "Abs":
                self.net = ai8x.FusedConv1dBNAbs(in_channels=num_channels,
                                                  out_channels=num_kernels,
                                                  kernel_size=kernel_size,
                                                  stride=stride,
                                                  padding=padding,
                                                  bias=True,
                                                  batchnorm="NoAffine",
                                                  **kwargs)
            else:
                self.net = ai8x.Conv1d(
                    in_channels=num_channels,
                    out_channels=num_kernels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    batchnorm=batchnorm,
                    **kwargs
                )
        else:
            self.net = ai8x.Conv1d(
                in_channels=num_channels,
                out_channels=num_kernels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                batchnorm=batchnorm,
                **kwargs
            )
        nn.utils.weight_norm(self.net.op.to(device))    
        nn.init.xavier_uniform_(self.net.op.weight)
        
    def forward(self, x):
        return self.net(x)

class Encoder(nn.Module):
    def __init__(self, 
                 n_channels=10, 
                 n_kernels=16, 
                 n_layers=3, 
                 seq_size=50,
                 device="cuda:0"):
        super(Encoder, self).__init__()
        self.feat_size = (seq_size-1) // 2**n_layers +1
        self.feat_dim = self.feat_size * n_kernels
        self.conv_stack = nn.Sequential(
            *([Conv1D(n_channels, n_kernels // 2**(n_layers-1), activation="ReLU", last=False, device=device)] +
              [Conv1D(n_kernels//2**(n_layers-l),
                         n_kernels//2**(n_layers-l-1), activation="ReLU", last=False, device=device)
               for l in range(1, n_layers-1)] +
              [Conv1D(n_kernels // 2, n_kernels, activation="ReLU", last=False, device=device)])
        )
    def forward(self, x):
        assert len(x.size())==3
        feats = self.conv_stack(x)
        return feats

class MLPLayer(nn.Module):
    def __init__(self, in_size, 
                 hidden_arch=[128/2, 512/2, 1024/2], 
                 output_size=None):
        
        super(MLPLayer, self).__init__()
        self.in_size = in_size
        self.output_size = output_size
        layer_sizes = [in_size] + [x for x in hidden_arch]
        self.layers = []
        
        for i in range(len(layer_sizes)-1):
            layer = ai8x.FusedLinearReLU(layer_sizes[i], layer_sizes[i+1])
            self.layers.append(layer)
           
        if output_size is not None:
            layer = ai8x.FusedLinearReLU(layer_sizes[-1], output_size)
            self.layers.append(layer)
            
        self.init_weights()
        self.mlp_network =  nn.Sequential(*self.layers)
        
    def forward(self, z):
        return self.mlp_network(z)
        
    def init_weights(self):
        for layer in self.layers:
            try:
                if isinstance(layer, ai8x.FusedLinearReLU):
                    nn.utils.weight_norm(layer)
                    nn.init.xavier_uniform_(layer.weight)
            except: pass
            

class AI85CNN1DNiLM(nn.Module):
    def __init__(self, in_size=1, 
                 output_size=5,
                 d_model=64,
                 dropout=0.1, 
                 seq_len=9,  
                 n_layers=5, 
                 n_quantiles=3, 
                 pool_filter=16,
                 device="cuda:0"):
        super(AI85CNN1DNiLM, self).__init__()

        self.dropout = nn.Dropout(dropout)
        self.enc_net = Encoder(n_channels=in_size, n_kernels=d_model, n_layers=n_layers, seq_size=seq_len, device=device)
        self.pool_filter = pool_filter
        self.mlp_layer = MLPLayer(in_size=d_model*pool_filter, hidden_arch=[1024], output_size=None)
        self.n_quantiles = n_quantiles
        
        self.fc_out_state  = ai8x.FusedLinearReLU(1024, output_size*2, bias=True)
        self.fc_out_power  = ai8x.FusedLinearReLU(1024, output_size*n_quantiles, bias=True)
        nn.init.xavier_normal_(self.fc_out_state.op.weight)
        nn.init.xavier_normal_(self.fc_out_power.op.weight)
        

    def forward(self, x):
        x = x.permute(0,2,1)
        B = x.size(0)
        print(f"X: max: {x.max()} min: {x.min()}")
        conv_out = self.dropout(self.enc_net(x))
        print(f"ENC_OUT: max: {conv_out.max()} min: {conv_out.min()}")
        conv_out = F.adaptive_avg_pool1d(conv_out, self.pool_filter)
        conv_out = conv_out.reshape(x.size(0), -1)
        print(f"AVG_POOL_OUT: max: {conv_out.max()} min: {conv_out.min()}")
        mlp_out  = self.dropout(self.mlp_layer(conv_out))
        print(f"MLP_OUT: max: {mlp_out.max()} min: {mlp_out.min()}")
        states_logits   = self.fc_out_state(mlp_out).reshape(B, 2, -1)
        power_logits    = self.fc_out_power(mlp_out)
        if self.n_quantiles>1:
            power_logits = power_logits.reshape(B, self.n_quantiles, -1)
        print(f"STATES: max: {states_logits.max()} min: {states_logits.min()}")
        print(f"POWER: max: {power_logits.max()} min: {power_logits.min()}")
        return states_logits, power_logits


In [14]:
model = AI85CNN1DNiLM(in_size=1, output_size=5, dropout=0.4, n_quantiles=len(quantiles), device=device, pool_filter=16)
model = model.to(device)

In [15]:
states, target = model(batch_select[0].to(device))

X: max: 12.040449142456055 min: -0.867415726184845
ENC_OUT: max: 1.6536457538604736 min: 0.0
AVG_POOL_OUT: max: 0.8706203103065491 min: 0.0
MLP_OUT: max: 0.5675764083862305 min: 0.0
STATES: max: 0.3692924678325653 min: 0.0
POWER: max: 0.42224159836769104 min: 0.0
